## Elsevier API with generated key with university account:

In [30]:
import requests
import csv

api_key = '02d0c4174287f162c45c7853a4ccd18e'


# Define the API endpoint and query parameters
api_url = 'https://api.elsevier.com/content/search/sciencedirect'
query_params = {
    # 'query': 'pub-date(2020-2022) AND random(1000)',
    'apiKey': api_key,
    'view': 'COMPLETE',
    'content': 'metadata',
    'start': 0,  # Start with the first result
    'count': 1000,  # Retrieve 1000 results
}

# Make the API request
response = requests.get(api_url, params=query_params) 

# Check the response status
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    publications = []
    for entry in data['search-results']['entry']:
        title = entry['dc:title']
        year = entry['prism:coverDisplayDate']
        abstract = entry['dc:description']
        citations = entry.get('citedby-count', 'N/A')
        publications.append([title, year, abstract, citations])

    # Save the data to a CSV file
    with open('publications.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write the header row
        csv_writer.writerow(['Title', 'Year', 'Abstract', 'Citations'])
        # Write the publication data
        csv_writer.writerows(publications)

    print("Data saved to 'publications.csv'")
else:
    print(f"API request failed with status code {response.status_code}")


API request failed with status code 405


Doesn't work:
"The 405 error happens when a client attempts to use an HTTP method that is not allowed by the server. For example, this issue can occur when a user tries to use a method such as PUT or DELETE on a resource that only allows GET or POST requests."

Not sure how to resolve this


Also 'query': 'pub-date(2020-2022) AND random(1000) does not work (that's why I commented it out), it gives a 401 error:
"The HyperText Transfer Protocol (HTTP) 401 Unauthorized response status code indicates that the client request has not been completed because it lacks valid authentication credentials for the requested resource."



In [33]:

import urllib3
import requests
import json
from bs4 import BeautifulSoup
from difflib import SequenceMatcher

from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import TimeoutException

In [39]:

myKeyList = ["7d2b7b28f29e2c6841427ef6dcb7aee0",
             "76b4641054b6e47c3d7f2703fc64035f",
             ]

class Citations:
    def __init__(self, pii):
        self.MY_API_KEY = myKeyList[0]
        self.pii = pii
        
        self.attrs = {
                    0:[]
                    }
        
        sciDir = self.getSciDir(pii)
        referList = self.getReferenceList(sciDir)
        print (referList)
        
    def __getitem__(self, key):
        if key in self.attrs:
            return self.attrs[key]
        
    def __setitem__(self, key, item):
        if key in self.attrs:
            self.attrs[key] = item
            
    def getSciDir(self, pii):
        resource = "http://api.elsevier.com/content/article/pii/"
        url = resource + pii
        
        webPage = requests.get(url)
        soup = BeautifulSoup(webPage.content.decode('utf-8'), "xml")
        
        try:
            sciDir = soup.find('link',{'rel':'scidir'}).get('href')
        except:
            sciDir = 'N/A'
        return sciDir
        
    def getReferenceList(self, sciDir):
        driver = webdriver.Firefox(executable_path=r'geckodriver.exe')
        driver.get(sciDir)
        driver.maximize_window()
        
        url = sciDir
        literatureList = []
        try:
            element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, 'references')))
            print ("Page Ready")
        except(TimeoutException):
            print ("Fail to load")
        else:
            soup = BeautifulSoup(driver.page_source.decode('utf-8'),"lxml")
            driver.quit()
            
            references = soup.find('dl', {'class':'references'})
            reference = references.findAll('dd', {'class':'reference'})
            
            
            for refer in reference:
#                 print refer
                try:
                    contribution = refer.find('div', {'class':'contribution'})
                    authors = contribution.getText().encode('utf-8', 'ignore')
                    try:
                        title = contribution.find('strong', {'class':'title'}).getText().encode('utf-8', 'ignore')
                    except:
                        title = ''
                    host = refer.find('div', {'class':'host'}).getText().encode('utf-8', 'ignore')                     
                except:
                    literature = refer.find('span').getText().encode('utf-8', 'ignore')
                else:
                    literature = authors + ', ' + title + ', ' + host
                
                print (literature)
                literatureList.append(literature)
        return literatureList

        
class Reference:
    #Science Direct Search
    def __init__(self, title):
        MY_API_KEY = myKeyList[1]
        self.title = title
        resource = "http://api.elsevier.com/content/search/scidir"
        
        url = (resource
          + "?query="
          + title)
#           + "&apiKey="
#           + self.myKeyList[0])
        
        resp = requests.get(url,
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
        
        print (('Input title') + str(title))
        
        results = json.loads(resp.text.encode('utf-8'))
        
        self.attrs = {
                      0: [None,  'identifier',               0],
                      1: [None,  'url',                      1],
                      2: [None,  'title',                    2],
                      3: [None,  'publicationName',          3],
                      4: [None,  'issueName',                4],
                      5: [None,  'issn',                     5],
                      6: [None,  'volume',                   6],
                      7: [None,  'issueIdentifier',          7],
                      8: [None,  'coverDisplayDate',         8],
                      9: [None,  'startingPage',             9],
                      10: [None,  'endingPage',              10],
                      11: [None,  'doi',                     11],
                      12: [None,  'pii',                     12],
                      13: [None,  'eid',                     13],
                      14: [None,  'authors',                 14]}
            
        refer = self.matchCheck(results)
        
        if refer != 'N/A':
            self.setAttr(refer)
                
        
    def __getitem__(self, key):
        if key in self.attrs:
            return self.attrs[key][0]
        
    def __setitem__(self, key, item):
        if key in self.attrs:
            self.attrs[key][0] = item
            
    def matchCheck(self, results):
        matchRefer = 'N/A'
        try:
            resultList = results['search-results']['entry']
        except(KeyError):
            pass
        else:
            for refer in resultList:
                similarity = self.similar(refer['dc:title'], self.title)
                print ('Similarity score = ' + str(similarity) + '(/1.0)')
                
                if  similarity> 0.8:
                    matchRefer = refer
                    print ("(matched to) "+str(matchRefer['dc:title'].encode('utf-8', 'ignore')))
                    break
        if matchRefer == 'N/A':
            print ("No matched reference")
        return matchRefer
    
    
    def similar(self, a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    def setAuthors(self, refer):
        authorsRaw = refer['authors']['author']
        authors = ''
        for indx, auth in enumerate(authorsRaw):
            authors += (auth['given-name']+' '
                        +auth['surname'])
            if indx != len(authorsRaw)-1:
                authors += ', '
        return authors
        
    
    def setAttr(self, refer):
        attrList = ['dc:identifier', 'prism:url', 'dc:title', 'prism:publicationName', 'prism:issueName', 'prism:issn', 'prism:volume', 'prism:issueIdentifier', 'prism:coverDisplayDate', 'prism:startingPage', 'prism:endingPage', 'prism:doi', 'pii', 'eid']
        
        for indx, attr in enumerate(attrList):
            try:
                refer[attr]
            except(KeyError):
                pass
            else:
                self.attrs[indx][0] = str(refer[attr]).encode('utf-8', 'ignore')

        try:
            author = self.setAuthors(refer)
        except(KeyError):
            pass
        else:
            self.attrs[14][0] = str(author).encode('utf-8', 'ignore')

In [44]:
import requests
from bs4 import BeautifulSoup
import csv
from difflib import SequenceMatcher

class Citations:
    def __init__(self, pii):
        self.MY_API_KEY = myKeyList[0]
        self.pii = pii
        self.attrs = {0: []}
        sciDir = self.getSciDir(pii)
        referList = self.getReferenceList(sciDir)
        print(referList)

    # Rest of your Citations class code...

class Reference:
    # Science Direct Search
    def __init__(self, title):
        MY_API_KEY = myKeyList[1]
        self.title = title
        resource = "http://api.elsevier.com/content/search/scidir"

        url = (resource + "?query=" + title)

        resp = requests.get(url, headers={'Accept': 'application/json', 'X-ELS-APIKey': MY_API_KEY})

        print(('Input title') + str(title))

        results = json.loads(resp.text.encode('utf-8'))

        self.attrs = {
            0: [None, 'identifier', 0],
            1: [None, 'url', 1],
            2: [None, 'title', 2],
            3: [None, 'publicationName', 3],
            4: [None, 'issueName', 4],
            5: [None, 'issn', 5],
            6: [None, 'volume', 6],
            7: [None, 'issueIdentifier', 7],
            8: [None, 'coverDisplayDate', 8],
            9: [None, 'startingPage', 9],
            10: [None, 'endingPage', 10],
            11: [None, 'doi', 11],
            12: [None, 'pii', 12],
            13: [None, 'eid', 13],
            14: [None, 'authors', 14]
        }

        refer = self.matchCheck(results)

        if refer != 'N/A':
            self.setAttr(refer)

    # Rest of your Reference class code...

def get_arxiv_publications():
    # Define the API endpoint and query parameters
    api_url = 'http://export.arxiv.org/api/query'
    query_params = {
        'search_query': 'cat:cs.AI',
        'start': 0,
        'max_results': 100,  # Get 100 publications
    }

    # Initialize a list to store publication data
    publications_2 = []

    # Make API requests in batches
    while query_params['start'] < query_params['max_results']:
        # Make the API request
        response = requests.get(api_url, params=query_params)

        # Check the response status
        if response.status_code == 200:
            # Parse the HTML response using BeautifulSoup with 'html.parser'
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract publication information
            for entry in soup.find_all('entry'):
                title = entry.find('title').text.strip()
                abstract = entry.find('summary').text.strip()

                # Check if 'arxiv:comment' element exists before accessing 'text'
                arxiv_comment = entry.find('arxiv:comment')
                if arxiv_comment:
                    citations = arxiv_comment.text.strip()
                else:
                    citations = 'N/A'

                # Append publication data to the list
                publications_2.append([title, abstract, citations])

            # Update the 'start' parameter for the next batch
            query_params['start'] += len(soup.find_all('entry'))
        else:
            print(f"API request failed with status code {response.status_code}")
            break

    return publications_2

def save_to_csv(data):
    # Specify the CSV file name
    csv_filename = 'publications_2.csv'

    # Write the data to the CSV file
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write the header row
        csv_writer.writerow(['Title', 'Abstract', 'Citations'])
        # Write the publication data
        csv_writer.writerows(data)

    print(f"Data saved to '{csv_filename}'")

def main():
    publications_2 = get_arxiv_publications()
    save_to_csv(publications_2)

if __name__ == "__main__":
    main()


c:\Users\49176\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
